In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#### CARGAMOS EL ARCHIVO INSTALLS CON FEATURES SACADOS DEL ARCHIVO INSTALLS

In [2]:
installs = pd.read_csv('TP 1er Cuatrimestre 2019/installsConFeatures.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,Unnamed: 0,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,early_morning,weekend,easter_holiday,nuevoDisositivo
created,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,94738,70,1,4432995619177048534,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,1,1
2019-04-18 00:00:01.851,94737,70,1,5904733559638204455,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,1,0
2019-04-18 00:00:05.152,111877,65,1,896373747754111825,False,True,6287817205707153877,6,102,2,...,2,2,1,0,0,8,0,False,1,0
2019-04-18 00:00:05.589,76146,27,0,3399210824535017892,False,False,6287817205707153877,0,190,2,...,21,14,6,1,1,2,0,False,1,1
2019-04-18 00:00:06.795,435184,339,0,1541425881979513687,False,False,6287817205707153877,0,190,2,...,68,46,10,1,1,6,0,False,1,1


### Dropeamos columnas que quedaron mal

In [3]:
installs.drop(['Unnamed: 0'],axis = 1,inplace=True)

#### Luego cargamos la primer ventana y sus lebels correspondientes a la segunda (Asi con todas las demas)

In [4]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [5]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)

In [6]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [7]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)

In [8]:
ventana_ultima = installs['2019-04-24':'2019-04-26'].reset_index()

In [9]:
ventana_ultima.head()

,created,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,early_morning,weekend,easter_holiday,nuevoDisositivo
0,2019-04-24 00:00:00.178,21,1,2105892908739159676,False,False,6287817205707153877,0,74,0,...,0,0,0,1,1,0,0,False,0,1
1,2019-04-24 00:00:02.223,226,1,208059911196607757,False,True,6287817205707153877,6,163,2,...,2,2,1,84,10,8,0,False,0,0
2,2019-04-24 00:00:02.352,70,1,9071467689792743380,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,0,0
3,2019-04-24 00:00:02.353,70,1,9071467689792743380,False,True,6287817205707153877,0,208,1095,...,0,0,0,28,6,9,0,False,0,1
4,2019-04-24 00:00:02.764,70,1,7588153492836870321,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,0,1


#### Como al algoritmo que aplicaremos despues no le gustan los NAN, a esos nan le ponemos 3 dias ( en segundos )

In [15]:
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
ventana_ultima = ventana_ultima.dropna()
label1 = label1.fillna(3*24*60*60)
label2 = label2.fillna(3*24*60*60)
label3 = label3.fillna(3*24*60*60)
label4 = label4.fillna(3*24*60*60)


#### Ahora sacaremos mas labels para cada ventana en particular...

In [ ]:
train1['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-18 00:00:00')).transform(lambda x: x.total_seconds())
train2['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-19 00:00:00')).transform(lambda x: x.total_seconds())
train3['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-20 00:00:00')).transform(lambda x: x.total_seconds())
train4['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-21 00:00:00')).transform(lambda x: x.total_seconds())
ventana_ultima['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-24 00:00:00')).transform(lambda x: x.total_seconds())


In [ ]:
train1['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train1))
train2['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train2))
train3['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train3))
train4['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train4))
ventana_ultima['accionesDesdeQueComenzoLaVentana'] = np.arange(len(ventana_ultima))

In [ ]:
train1.drop(['created'],axis =1, inplace = True )
train2.drop(['created'],axis =1, inplace = True )
train3.drop(['created'],axis =1, inplace = True )
train4.drop(['created'],axis =1, inplace = True )
ventana_ultima.drop(['created'],axis =1, inplace = True )


In [ ]:
#### XGBOOST LE GUSTAN LAS COLUMNAS ORDENADAS... 
ventana_ultima=ventana_ultima.sort_index(axis=1, ascending=False)
train1=train1.sort_index(axis=1, ascending=False)
train2=train2.sort_index(axis=1, ascending=False)
train3=train3.sort_index(axis=1, ascending=False)
train4=train4.sort_index(axis=1, ascending=False)



In [ ]:

train1.head()

#### Ahora comenzaremos a entrenar con XGBOOST

In [ ]:
train1.shape,label1.shape

In [ ]:
train1.columns

In [ ]:
train1 = train1.drop('wifi',axis=1,inplace=False)
train2 = train2.drop('wifi',axis=1,inplace=False)
train3 = train3.drop('wifi',axis=1,inplace=False)
train4 = train4.drop('wifi',axis=1,inplace=False)
ventana_ultima = ventana_ultima.drop('wifi',axis=1,inplace=False)

In [ ]:
data_dmatrix = xgb.DMatrix(data=train,label=label1)


In [ ]:
xg_reg1 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(train1, label1, test_size=0.2, random_state=123)

In [ ]:
xg_reg1.fit(X_train,y_train)

In [ ]:
predicted1 = xg_reg1.predict(X_test)
predicted1 = pd.DataFrame(predicted1)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted1[0])**2)/len(y_train)))
RMSE

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(train2, label2, test_size=0.2, random_state=123)

In [ ]:
xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg2.fit(X_train,y_train)

In [ ]:
predicted2 = xg_reg2.predict(X_test)
predicted2 = pd.DataFrame(predicted2)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted2[0])**2)/len(y_train)))
RMSE

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(train3, label3, test_size=0.2, random_state=123)

In [ ]:
xg_reg3 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg3.fit(X_train,y_train)

In [ ]:
predicted3 = xg_reg3.predict(X_test)
predicted3 = pd.DataFrame(predicted3)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted3[0])**2)/len(y_train)))
RMSE

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(train4, label4, test_size=0.2, random_state=123)

In [ ]:
xg_reg4 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg4.fit(X_train,y_train)

In [ ]:
predicted4 = xg_reg4.predict(X_test)
predicted4 = pd.DataFrame(predicted4)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted4[0])**2)/len(y_train)))
RMSE

# Voting

In [ ]:
target = pd.read_csv('TP 1er Cuatrimestre 2019/final.csv', low_memory=False, names=['ref_hash'])
#target = pd.read_csv('data/final.csv', low_memory=False, names=['ref_hash'])

modelos = [model1,model2,model3,model4]
predictions = [0 for y in range(len(modelos))] 

for i,modelo in enumerate(modelos):
    prediction = modelo.predict(train5)
    ventana_ultima['obj'] = prediction
    mergeados = ventana_ultima[['ref_hash','obj']].merge(target, left_on = 'ref_hash',right_on='ref_hash', how='right')
    mergeados = mergeados.fillna(60*60*24*3)
    mergeados = mergeados.groupby('ref_hash').agg({'obj':'mean'})
    predictions[i] = mergeados.reset_index()

predictions[0] = predictions[0].rename(columns={'obj':'obj1'})
predictions[1] = predictions[1].rename(columns={'obj':'obj2'})
predictions[2] = predictions[2].rename(columns={'obj':'obj3'})
predictions[3] = predictions[3].rename(columns={'obj':'obj4'})

len(predictions[0])

#predictions

pred = ((predictions[0].merge(predictions[1],on='ref_hash',how='outer')).merge(predictions[2],on='ref_hash',how='inner')).merge(predictions[3],on='ref_hash',how='inner')

pred.head()

### Averaging

pred['obj'] = (pred['obj1']+pred['obj2']+pred['obj3']+pred['obj4'])/4

averaging = pred[['ref_hash','obj']]
averaging.head()

averaging.info()

averaging.to_csv('data/preds_xgboost_conFeatures_averaging_v1.csv',index=False)

### Weighted Averaging

pred['obj'] = pred['obj1']*0.1+pred['obj2']*0.2+pred['obj3']*0.3+pred['obj4']*0.4

weighted = pred[['ref_hash','obj']]
weighted.head()

weighted.info()

weighted.to_csv('data/preds_xgboost_conFeatures_weighted_v1.csv',index=False)